In [5]:
%%capture
%pip install websockets==8.1

In [6]:
from mep_client.k3_mep_client import K3MepClient

client = K3MepClient()

In [7]:
import time

def update_variables():
    variable_output_widget.clear_output()
    with variable_output_widget:
        print()
        print()
        print("Variables")
        print("―――――――――")
        client.variables()
        
def update_stack():
    stack_output_widget.clear_output()
    with stack_output_widget:
        print()
        print()
        print("Stack Trace")
        print("―――――――――――")
        client.stack_trace()
        
def update_model():
    model_output_widget.clear_output()
    with model_output_widget:
        print()
        print()
        print("Model")
        print("―――――")
        client.source(line_numbers=True)

def connection_button_click(b):
    with output_widget:
        client.connect(host=host_widget.value, port=port_widget.value, endpoint=endpoint_widget.value)
        client.initialize()

def launch_button_click(b):
    simulation_output_widget.clear_output()
    with simulation_output_widget:
        print()
        print()
        print("Simulation Output")
        print("―――――――――――――――――")
        client.launch(model_uri=model_uri_widget.value, model_entry_point=model_entrypoint_widget.value,
                     init_method=init_method_widget.value, init_args=init_args_widget.value, method_entry_point=method_entrypoint_widget.value)
    update_model()
    update_stack()
    update_variables()
        
def continue_button_click(b):
    with simulation_output_widget:
        client.continue_()
        while client.simulation_running and not client.paused:
            time.sleep(0.1) # Required to capture all outputs from thread
    update_model()
    update_stack()
    update_variables()
    
def next_button_click(b):
    with simulation_output_widget:
        client.next()
    update_model()
    update_stack()
    update_variables()
            
def step_in_button_click(b):
    with simulation_output_widget:
        client.step_in()
    update_model()
    update_stack()
    update_variables()
    
def step_out_button_click(b):
    with simulation_output_widget:
        client.step_out()
    update_model()
    update_stack()
    update_variables()
    
def terminate_button_click(b):
    with output_widget:
        client.terminate()

def restart_button_click(b):
    simulation_output_widget.clear_output()
    with simulation_output_widget:
        print()
        print()
        print("Simulation Output")
        print("―――――――――――――――――")
        client.restart()
    update_model()
    update_stack()
    update_variables()
            
def set_breakpoints_button_click(b):
    with output_widget:
        int_lines = []
        lines = set_breakpoints_lines_widget.value.split()
        for line in lines:
            int_lines.append(int(line))
        client.set_breakpoints(int_lines)

In [8]:
import ipywidgets

host_widget = ipywidgets.Text(value='localhost', description='Host:')
port_widget = ipywidgets.IntText(value=8090, description='Port:')
endpoint_widget = ipywidgets.Text(value='mep/', description='Endpoint:')
connection_button = ipywidgets.Button(description='Connect')
connection_button.on_click(connection_button_click)
connection_widgets = ipywidgets.VBox([host_widget, port_widget, endpoint_widget, connection_button])

model_uri_widget = ipywidgets.Text(value='', description='Model URI:', layout=ipywidgets.Layout(min_width='50%'), style={'description_width': '150px'})
model_entrypoint_widget = ipywidgets.Text(value='/', description='Model Entrypoint:', layout=ipywidgets.Layout(min_width='50%'), style={'description_width': '150px'})
init_method_widget = ipywidgets.Text(value='', description='Initialization Method:', layout=ipywidgets.Layout(min_width='50%'), style={'description_width': '150px'})
init_args_widget = ipywidgets.Text(value='', description='Initialization Arguments:', layout=ipywidgets.Layout(min_width='50%'), style={'description_width': '150px'})
method_entrypoint_widget = ipywidgets.Text(value='', description='Method Entrypoint:', layout=ipywidgets.Layout(min_width='50%'), style={'description_width': '150px'})
launch_button = ipywidgets.Button(description='Launch')
launch_button.on_click(launch_button_click)
launch_widgets = ipywidgets.VBox([model_uri_widget, model_entrypoint_widget, init_method_widget, method_entrypoint_widget, init_args_widget, launch_button])

accordion = ipywidgets.Accordion(children=[connection_widgets, launch_widgets])
accordion.set_title(0, 'Connection Parameters')
accordion.set_title(1, 'Launch Configuration')

output_widget = ipywidgets.Output()
model_output_widget = ipywidgets.Output()
simulation_output_widget = ipywidgets.Output()
stack_output_widget = ipywidgets.Output()
variable_output_widget = ipywidgets.Output()

continue_button = ipywidgets.Button(description='▶', tooltip='Continue', layout=ipywidgets.Layout(width='50px'))
continue_button.on_click(continue_button_click)
next_button = ipywidgets.Button(description='➡', tooltip='Next', layout=ipywidgets.Layout(width='50px'))
next_button.on_click(next_button_click)
step_in_button = ipywidgets.Button(description='  ➡】', tooltip='Step In', layout=ipywidgets.Layout(width='50px'))
step_in_button.on_click(step_in_button_click)
step_out_button = ipywidgets.Button(description='【➡  ', tooltip='Step Out', layout=ipywidgets.Layout(width='50px'))
step_out_button.on_click(step_out_button_click)
terminate_button = ipywidgets.Button(description='⏹', tooltip='Terminate', layout=ipywidgets.Layout(width='50px'))
terminate_button.on_click(terminate_button_click)
restart_button = ipywidgets.Button(description='↻', tooltip='Restart', layout=ipywidgets.Layout(width='50px'))
restart_button.on_click(restart_button_click)
set_breakpoints_lines_widget = ipywidgets.Text(description='Breakpoints', placeholder='line1 line2 ...')
set_breakpoints_button = ipywidgets.Button(description='Set', tooltip='Set Breakpoints', layout=ipywidgets.Layout(width='50px'))
set_breakpoints_button.on_click(set_breakpoints_button_click)
debug_widgets = ipywidgets.VBox([ipywidgets.HBox([continue_button, next_button, step_in_button, step_out_button, terminate_button, restart_button]),
                                 ipywidgets.HBox([set_breakpoints_lines_widget, set_breakpoints_button])])

ipywidgets.VBox([accordion, output_widget, model_output_widget, simulation_output_widget, variable_output_widget, stack_output_widget, debug_widgets])